The [Containers](Containers.ipynb) Tutorial introduced the [HoloMap](Containers.ipynb#HoloMap), a core HoloViews datastructure that allows easy exploration of parameter spaces. The essence of a HoloMap is that it contains a collection of [Elements](Elements.ipynb) that you can easily visualize and slice.

HoloMaps are containers that hold elements at sampled points in a multidimensional space. Although this makes them useful for exploring high-dimensional parameter spaces, they can very quickly consume huge amounts of memory as a result. For instance, a hundred samples along four orthogonal dimensions would need a HoloMap containing a hundred *million* elements. This highlights some of the limitations of ``HoloMaps``:

* HoloMaps may require the generation of millions of elements before they are fully defined.
* HoloMaps can easily exhaust all the memory available to Python.
* HoloMaps can simultaneously exhaust all the memory in the browser when displayed.
* Static export of a notebook containing HoloMaps can result in impractically large HTML files.

The ``DynamicMap`` addresses these issues by computing and displaying elements dynamically, allowing exploration of much larger datasets:

* DynamicMaps generate elements on the fly allowing the process of exploration to begin immediately.
* DynamicMaps do not require fixed sampling, allowing exploration of parameters with arbitrary resolution.
* DynamicMaps are lazy in the sense they only compute only as much data as the user wishes to explore.

The limitations of ``DynamicMaps`` are:

* DynamicMaps require a live notebook server and cannot be directly exported to static HTML.
* DynamicMaps do not hold data which reduces the utility of a pickling DynamicMap.

Now we have enumerated the pros and cons of DynamicMaps, let's create one!

<center><div class="alert alert-info" role="alert">To use visualize and use a **DynamicMap** you need to be running a live Jupyter server.<br>This tutorial assumes that it will be run in a live notebook environment.</div></center>

# ``DynamicMap``  <a id='DynamicMap'></a>

Let's start by importing HoloViews and loading the extension:

In [ ]:
import holoviews as hv
import numpy as np
hv.notebook_extension()

We will now create the ``DynamicMap`` equivalent of the ``HoloMap`` introduced in the [Containers Tutorial](Containers.ipynb#HoloMap). The ``HoloMap`` in that tutorial consisted of ``Image`` elements containing sine ring arrays s defined by the ``sine_array`` function:

In [ ]:
x,y = np.mgrid[-50:51, -50:51] * 0.1

def sine_array(phase, freq):
    return np.sin(phase + (freq*x**2+freq*y**2))

This function returns NumPy arrays when called:

In [ ]:
sine_array(0,1).shape

For a ``DynamicMap`` we will need a function that returns HoloViews elements. This is easy as a trivial extension of the ``sine_array`` function allows it to output [Image](Elements.ipynb#Image) elements:

In [ ]:
def sine_image(phase, freq):
    return hv.Image(np.sin(phase + (freq*x**2+freq*y**2)))

In [ ]:
sine_image(0,1)

Now we can demonstrate the first type of exploration enabled by a ``DynamicMap`` we call 'bounded' mode.

## Bounded mode

A 'closed' mode ``DynamicMap`` is simply one where all the key dimensions have finite bounds. Bounded mode has the following properties:

* The limits of the space that can be explored must be declared for all the key dimensions.
* You can explore within the declared bounds at any resolution.
* The ``DynamicMap`` is defined using a callable that *must* be a function of its arguments (i.e the output is strictly determined by the input arguments).

We can now create a DynamicMap by simply declaring the ranges of the two dimensions and passing the ``sine_image`` function as the ``.data``:



In [ ]:
dmap = hv.DynamicMap(sine_image, kdims=[hv.Dimension('phase',range=(0, np.pi)),
                                 hv.Dimension('frequency', range=(0.01,np.pi))])

This object is created instantly as no data has been generated yet. We can now look at the repr of this object: 

In [ ]:
repr(dmap)

All ``DynamicMaps`` will look similar, only differing in the listed dimensions. Now let's see how this dynamic map visualizes itself:

In [ ]:
dmap

If you are running this tutorial in a live notebook, you should now see something that looks like the ``HoloMap`` in the [Containers Tutorial](Containers.ipynb#HoloMap). There are some key differences though:

* You can now pick any value of **phase** or **frequency** up to the precision allowed by the slider.
* What you see in the cell above will not be exported in any HTML snapshot of the notebook.

### Using your own callable

You can use any callable to define a ``DynamicMap`` in closed mode. A valid ``DynamicMap`` is defined by the following criteria:

* There must be as many positional arguments in the callable signature as key dimensions.
* The argument order in the callable signature must match the order of the declared key dimensions.
* All key dimensions are defined with a bounded ``range`` or ``values`` parameter (for categorical dimensions).